In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re

# 참고한 코드: https://bumcrush.tistory.com/116 [맑음때때로 여름]

In [10]:
result = crawler(89)

1
11
21
31
41
51
61
71
81
91
101
111
121
131
141
151
161
171
181
191
201
211
221
231
241
251
261
271
281
291
301
311
321
331
341
351
361
371
381
391
401
411
421
431
441
451
461
471
481
491
501
511
521
531
541
551
561
571
581
591
601
611
621
631
641
651
661
671
681
691
701
711
721
731
741
751
761
771
781
791
801
811
821
831
841
851
861
871
881


In [12]:
result.to_csv('20200101_20200930_stalking_naver.csv', encoding='utf-8')

In [9]:
def crawler(keyword, maxpage, startdate, enddate):
    '''
    keyword: (string) 검색어
    maxpage: (int) 크롤링할 총 페이지 수
    startdate: (string)(2020.01.01) 검색할 시작 날짜
    enddate: (string)(2020.01.31) 검색할 마지막 날짜
    '''
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1 # 11= 2페이지 21=3페이지 31=4페이지 ...81=9페이지 , 91=10페이지, 101=11페이지
    final_result = pd.DataFrame(columns=['date', 'title', 'source', 'contents', 'link'])
    while page <= maxpage_t:
        # field=1 제목 photo=3 지면기사 
        # sort=1 최신순 sort=2 오래된순
        url = "https://search.naver.com/search.naver?where=news&query="\
              + keword\
              + "&sm=tab_pge&sort=2&photo=0&field=1&reporter_article=&pd=3&ds="\
              + startdate\
              + "&de="\
              + enddate\
              + "&docid=&nso=so:dd,p:from"\
              + startdate.replace('.', '')\
              + "to"\
              + enddate.replace('.', '')\
              +",a:t&mynews=0&start=" + str(page) + "&refresh_start=0"
        response = requests.get(url)
        html = response.text
        
        #뷰티풀소프의 인자값 지정
        soup = BeautifulSoup(html, 'html.parser')

        #태그에서 제목과 링크주소 추출
        atags = soup.select('._sp_each_title')
        
        title_text = []
        link_text = []
        source_text = []
        date_text = []
        contents_text = []
        
        for atag in atags:
            title_text.append(atag.text) #제목
            link_text.append(atag['href']) #링크주소

        #신문사 추출
        source_lists = soup.select('._sp_each_source')
        for source_list in source_lists:
            source_text.append(source_list.text) #신문사

        #날짜 추출
        date_lists = soup.select('.txt_inline')
        for date_list in date_lists:
            test = date_list.text
            date_cleansing(test, date_text) #날짜 정제 함수사용
            
        #본문요약본
        contents_lists = soup.select('ul.type01 dl')
        for contents_list in contents_lists:
            #print('==='*40)
            #print(contents_list)
            contents_cleansing(contents_list, contents_text) #본문요약 정제화
            
        #모든 리스트 딕셔너리형태로 저장
        result= {"date" : date_text , "title":title_text , "source" : source_text ,"contents": contents_text ,"link":link_text }
        print(page)
        
        df = pd.DataFrame(result) #df로 변환
        page += 10
        
        final_result = pd.concat([final_result, df], ignore_index=True)
    return final_result
#출처: https://bumcrush.tistory.com/116 [맑음때때로 여름]

In [4]:
#날짜 정제화 함수
def date_cleansing(test, date_text):
    try:
        #지난 뉴스
        #머니투데이 10면1단 2018.11.05. 네이버뉴스 보내기
        pattern = '\d+.(\d+).(\d+).' #정규표현식
        r = re.compile(pattern)
        match = r.search(test).group(0) # 2018.11.05.
        date_text.append(match)
    except AttributeError:
        #최근 뉴스
        #이데일리 1시간 전 네이버뉴스 보내기
        pattern = '\w* (\d\w*)' #정규표현식
        r = re.compile(pattern)
        match = r.search(test).group(1)
        #print(match)
        date_text.append(match)

# 출처: https://bumcrush.tistory.com/116 [맑음때때로 여름]

In [5]:
#내용 정제화 함수
def contents_cleansing(contents, contents_text):
    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '',
    str(contents)).strip() #앞에 필요없는 부분 제거
    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '',
    first_cleansing_contents).strip()#뒤에 필요없는 부분 제거 (새끼 기사)
    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
    contents_text.append(third_cleansing_contents)
    #print(contents_text)

# 출처: https://bumcrush.tistory.com/116 [맑음때때로 여름]